In [42]:
import pandas as pd

In [43]:
df = pd.read_csv('0. merged_df.csv')

In [44]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [45]:
lon_min, lon_max = 11.5, 13
lat_min, lat_max = 45, 46

In [46]:
df = df.loc[(df.lon >= lon_min) & (df.lon <= lon_max)]
df = df.loc[(df.lat >= lat_min) & (df.lat <= lat_max)]
df

,osm_id,code,fclass,name,lon,lat,SELECT
0,4710535,7218,grass,NaN,12.272857,45.503580,nature
1,4710570,7218,grass,NaN,12.279067,45.505727,nature
2,4710648,7206,cemetery,Cimitero di Favaro Veneto,12.280370,45.511354,nature
3,4851260,7213,nature_reserve,Caserma Edmondo Matter,12.240170,45.505275,nature
4,4851276,7206,cemetery,Cimitero di Mestre,12.243382,45.501909,nature
...,...,...,...,...,...,...,...
2662743,1095491695,8200,water,NaN,11.902024,45.810205,nature
2662744,1095491696,8200,water,NaN,11.905475,45.812606,nature
2662745,1095603673,8200,water,NaN,11.797104,45.924658,nature
2662746,1095603675,8200,water,NaN,11.789369,45.927601,nature


In [47]:
n_rows, n_columns = 200, 200

lat_min, lat_max = df.lat.min(), df.lat.max()  # rows
lon_min, lon_max = df.lon.min(), df.lon.max()  # columns

diff_lat = lat_max - lat_min
diff_lon = lon_max - lon_min

step_lat = diff_lat / n_rows
step_lon = diff_lon / n_columns

In [48]:
categories = df['SELECT'].drop_duplicates().to_list()

In [1]:
starting_point_lat, starting_point_lon = lat_min, lon_min # start from the highest column and the first column

result_log = {}

#starting bbox
bbox_lat_min, bbox_lat_max = starting_point_lat, starting_point_lat + step_lat
bbox_lon_min, bbox_lon_max = starting_point_lon, starting_point_lon + step_lon

for row in range(n_rows):
    # reset starting point of the column axis
    bbox_lon_max = starting_point_lon
    bbox_lon_min = starting_point_lon + step_lon

    row_df = df.loc[(df.lat > bbox_lat_min) & (df.lat <= bbox_lat_max), :]

    for column in range(n_columns):
        cell_df = row_df.loc[(row_df.lon > bbox_lon_min) & (row_df.lon <= bbox_lon_max), :]

        cell_name = 'R' + str(row + 1) + 'C' + str(column + 1)
        cell_values = [row + 1, column +1, bbox_lat_min, bbox_lat_max, bbox_lon_min, bbox_lon_max]

        for category in categories:
            try:
                count = cell_df.SELECT.value_counts()[category]
                cell_values.append(count)

            except:
                cell_values.append(0)

        result_log[cell_name] = cell_values

        #print('LAT:', bbox_lat_min, '   ', bbox_lat_max, '  LON:', bbox_lon_min, '   ', bbox_lon_max, '    COUNT:', cell_df.shape[0])

        bbox_lon_min = bbox_lon_max
        bbox_lon_max += step_lon

    bbox_lat_min = bbox_lat_max
    bbox_lat_max += step_lat

NameError: name 'lat_min' is not defined

In [50]:
result_df = pd.DataFrame(result_log).T
result_df.columns = ['row', 'column', 'lat_min', 'lat_max', 'lon_min', 'lon_max'] + categories
result_df

,row,column,lat_min,lat_max,lon_min,lon_max,nature,entartainment,transports,art,sport,tourism,security,accessibility
R1C1,1.0,1.0,45.000004,45.005004,11.507500,11.500001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C2,1.0,2.0,45.000004,45.005004,11.500001,11.507500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
R1C3,1.0,3.0,45.000004,45.005004,11.507500,11.515000,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
R1C4,1.0,4.0,45.000004,45.005004,11.515000,11.522500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R1C5,1.0,5.0,45.000004,45.005004,11.522500,11.530000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R200C196,200.0,196.0,45.994998,45.999998,12.955000,12.962500,3.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0
R200C197,200.0,197.0,45.994998,45.999998,12.962500,12.970000,4.0,0.0,11.0,1.0,3.0,0.0,0.0,0.0
R200C198,200.0,198.0,45.994998,45.999998,12.970000,12.977500,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
R200C199,200.0,199.0,45.994998,45.999998,12.977500,12.985000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [53]:
result_df.to_csv('1. complete_df.csv')

In [51]:
filtered_df = result_df[result_df[categories].T.sum(0) != 0] # remove the cells that have 0 on every fclass
filtered_df

,row,column,lat_min,lat_max,lon_min,lon_max,nature,entartainment,transports,art,sport,tourism,security,accessibility
R1C2,1.0,2.0,45.000004,45.005004,11.500001,11.5075,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
R1C3,1.0,3.0,45.000004,45.005004,11.507500,11.5150,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
R1C5,1.0,5.0,45.000004,45.005004,11.522500,11.5300,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
R1C6,1.0,6.0,45.000004,45.005004,11.530000,11.5375,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
R1C7,1.0,7.0,45.000004,45.005004,11.537500,11.5450,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R200C196,200.0,196.0,45.994998,45.999998,12.955000,12.9625,3.0,1.0,5.0,0.0,0.0,0.0,0.0,1.0
R200C197,200.0,197.0,45.994998,45.999998,12.962500,12.9700,4.0,0.0,11.0,1.0,3.0,0.0,0.0,0.0
R200C198,200.0,198.0,45.994998,45.999998,12.970000,12.9775,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
R200C199,200.0,199.0,45.994998,45.999998,12.977500,12.9850,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [52]:
filtered_df.to_csv('1. filtered_df.csv')